# УРОВЕНЬ INTERN

## 2. SMAPE

* MAPE (Mean Absolute Percentage Error) — метрика ошибки, учитывающая средний процент отклонения предсказываемого значения от реального.

* sMAPE (от symmetric MAPE) — логическое продолжение MAPE, где в знаменателе стоит уже сумма модулей предсказания и факта, а не только факта

In [16]:
import numpy as np

def smape(y_true: np.array, y_pred: np.array) -> float:
    denominator = np.abs(y_true) + np.abs(y_pred)
    den = np.where(denominator > 0, 1, denominator)
    return np.mean(2 * np.abs(y_true - y_pred) / den )

In [17]:
y_true = np.array(0.5)
y_pred = np.array(50)
a = smape(y_true, y_pred)
a

99.0

## 3. VALID EMAILS

https://docs.python.org/3/library/re.html

Можно ускорить код засчет предварительной компиляции выражения

In [6]:
import re
from typing import List
import time
import numpy as np



def valid_emails(strings: List[str]) -> List[str]:
    """Take list of potential emails and returns only valid ones"""

    valid_email_regex = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$"
    
    def is_valid_email(email: str) -> bool:
        return bool(re.fullmatch(valid_email_regex, email))
    
    start = time.perf_counter()
    emails = []
    for email in strings:
        if is_valid_email(email):
            emails.append(email)
            
    finish  = time.perf_counter() 
    
    print(f"Вычисление заняло {start - finish:0.4f} секунд")         

    return emails


In [7]:
def valid_emails_imrove(strings: List[str]) -> List[str]:
    """Take list of potential emails and returns only valid ones"""
    valid_email_regex = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$"
    valid = re.compile(valid_email_regex)
    
    def is_valid_email(email: str) -> bool:
        return bool(valid.fullmatch(email))
    
    start = time.perf_counter()
    emails = []
    for email in strings:
        if is_valid_email(email):
            emails.append(email)
    finish  = time.perf_counter()               
    print(f"Вычисление заняло {start - finish:0.4f} секунд")
    return emails

In [20]:
wrong = list(np.full(1000, 'sbsbfs@7hbhcw3'))
right = list(np.full(10, 'sfsd@mail.ru'))
emails = wrong+right

In [33]:
valid_emails(emails)

Вычисление заняло -0.0022 секунд


['sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru']

In [38]:
valid_emails_imrove(emails)

Вычисление заняло -0.0009 секунд


['sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru',
 'sfsd@mail.ru']

## 4. Cold start
Возьмем данные по количеству проданных товаров из задачи "SKU SALES" и заполним пропуски средним значением по имеющимся данным

In [5]:
import pandas as pd
import numpy as np

df = pd.read_excel('sku_sales.xlsx')
df.head()

,days,sku
0,2020-06-08,5
1,2020-06-10,5
2,2021-01-01,378
3,2021-01-02,1020
4,2021-01-03,2581


In [6]:
# Заменили все значения продаж в январе 2021 на na для проверки работоспособности функции
df.loc[df['days'].str.contains('2021-01'), 'sku'] = None

# Создаем столбец для дальнейшей группировки и подсчета среднего
df['day'] = df.days.apply(lambda x: x.split('-')[2])

In [7]:
import pandas as pd
import numpy as np


def fillna_with_mean(
    df: pd.DataFrame, target: str, group: str
) -> pd.DataFrame:
    df_new = df.copy() #важно не забывать делать копию, иначе изменим изначальный датафрейм
    group = df_new.groupby(group)[target].transform('mean')
    df_new.loc[df[target].isnull(), target] = group.apply(np.floor)
    return df_new


In [8]:
df_new = fillna_with_mean(df, 'sku', 'day')
df_new

,days,sku,day
0,2020-06-08,5.0,08
1,2020-06-10,5.0,10
2,2021-01-01,9343.0,01
3,2021-01-02,10509.0,02
4,2021-01-03,10308.0,03
...,...,...,...
361,2021-12-26,1081.0,26
362,2021-12-27,12879.0,27
363,2021-12-28,1588.0,28
364,2021-12-29,14686.0,29


In [9]:
df

,days,sku,day
0,2020-06-08,5.0,08
1,2020-06-10,5.0,10
2,2021-01-01,NaN,01
3,2021-01-02,NaN,02
4,2021-01-03,NaN,03
...,...,...,...
361,2021-12-26,1081.0,26
362,2021-12-27,12879.0,27
363,2021-12-28,1588.0,28
364,2021-12-29,14686.0,29


## 5. Stocks

* SKU (Stock Keeping Unit), уникальный ID товара (тип int)
* GMV (Gross Merchandise Volume), аналог розничного товарооборота (тип float)
* stock – число единиц товара на складе (тип int)
* price – цена на товар (тип float)

**Notes:**
'/' - деление, '//' -целочисленное деление; '/' и округление работает быстрее

Ex.: 5 / 2 will return 2.5 and 5 // 2 will return 2

In [2]:
import pandas as pd
df_p = pd.read_excel('stocks.xlsx')
df_p

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5


In [3]:
def limit_gmv(df: pd.DataFrame):
    df.loc[df.price*df.stock < df.gmv, 'gmv'] = df.price*df.stock
    df.loc[df.gmv%df.price != 0, 'gmv'] = round(df.gmv/df.price)*df['price']     
    return df

In [4]:
limit_gmv(df_p)

,sku,gmv,price,stock
0,100,300,100,3
1,200,350,70,10
2,300,480,120,5


In [11]:
def limit_gmv(df: pd.DataFrame):
    """Принимает на вход датафрейм с предсказанием оборота,
        возвращает обработанные датафрейм"""
    df = df.copy()
    df['gmv'] = ((df.gmv/df.price).astype(int) * df['price']) \
                                .clip(upper = df.price*df.stock) 
    return df

In [12]:
limit_gmv(df_p)

,sku,gmv,price,stock
0,100,300,100,3
1,200,350,70,10
2,300,480,120,5
